В данном обзоре мы рассмотрим два подхода к обучению языковых моделей на основе трансформеров:

- GPT (статья "[Improving Language Understanding by Generative Pre-Training](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf)", 2018). GPT в дальнейшем развитие в виде [GPT-2]($Language Models are Unsupervised Multitask Learners$) и далее [GPT-3]($Language Models are Few-Shot Learners$), которая стала способна генерировать тексты, [почти неотличимые от написанных человеком](https://twitter.com/raphamilliere/status/1289129723310886912).

- BERT (статья "[BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805)", 2018).

Создание BERT и GPT многие упоминают как "ImageNet moment" в области NLP, проводя аналогию с созданием [AlexNet]($ImageNet Classification with Deep Convolutional Neural Networks$), давшей толчок развитию компьютерного зрения в 2012 году.

### Self-supervised learning в NLP

В большинстве задач, в том числе в задачах NLP (обработки естественного языка), неразмеченных данных доступно на порядки больше, чем размеченных. Неразмеченные данные в NLP - это любые тексты (из книг, Википедии, новостных сайтов и так далее). Можно собрать сотни гигабайт таких текстов. Общая идея подхода, называемого предобучением без учителя (unsupervised pretraining) в том, чтобы каким-то образом использовать для предобучения большое количество неразмеченных данных. Мы надеемся, что обучившись на таких данных, модель научится понимать структуру языка. Такую модель затем можно "дообучить" для решения какой-то конкретной задачи.

Технически нейронные сети требуют для обучения размеченные данные, то есть набор пар "вопрос + эталонный ответ". Каким же образом можно обучать их на неразмеченных данных? Для этого можно использовать подход, называемый self-supervised learning, то есть автоматическая генерация пар "вопрос + эталонный ответ" из неразмеченных данных. Сначала рассмотрим примеры не из области NLP:

- **Пример 1.** Автоэнкодеры. Выходные данные равны входным, то есть на выходе сеть должна выдать то же самое, что получила на вход. В качестве альтернативы на вход можно подавать искаженные данные, например изображение с наложенным шумом, а эталонным ответом (разметкой) считать исходное изображение.

- **Пример 2.** Задача in-painting в компьютерном зрении. На вход подается изображение с вырезанным участком (как в cutout), а на выходе должна выдать исходное изображение.

Теперь рассмотрим применение этого подхода на текстовых данных:

- **Вариант 1.** Мы выбираем из случайного текста случайный участок небольшого размера и удаляем в нем центральное слово. На вход модель получает участок без центрального слова, и на выходе должна определить, какое слово было удалено. Либо, наоборот, получив на вход центральное слово, нужно предсказать контекст.

- **Вариант 2.** *Masked lauguage modeling, или [Cloze task](https://en.wikipedia.org/wiki/Cloze_test)* - близкий аналог задачи in-painting. В тексте определенный процент слов заменяется на токен [MASK]. Сеть должна определить, какие слова стояли на месте [MASK]. Технически это реализуется таким образом: для каждого токена [MASK] сеть выдает распределение вероятностей по всем словам в словаре. Мы знаем удаленное слово, то есть имеем эталонный ответ. В качестве функции потерь можно применить категориальную кроссэнтропию между выданным распределением вероятностей и эталонным ответом. То есть данная задача решается как задача классификации.

- **Вариант 3.** *Next sentence prediction*. Входные данные составлены из двух предложений, соединенных токеном [SEP]. Сеть должна определить, стояли ли эти предложения друг за другом в исходном тексте, или же взяты из разных частей текста. То есть сеть решает задачу бинарной классификации.

- **Вариант 4.** Получив на вход незаконченный текст, сеть должна определить, какое слово стоит следующим.

Перечисленные задачи называются задачами языкового моделирования. Они имеют общую черту: верный ответ может быть далеко не единственным. Например, из текста вырезан участок, в котором пропущено имя. Если это имя больше не встречается в вырезанном участке, то сети неоткуда брать нужную информацию. Либо некое пропущенное слово можно заменить синонимом, и по смыслу такой ответ можно тоже считать верным. Но, тем не менее, метод self-supervised learning хорошо работает, поскольку используется очень много данных, и для минимизации loss модели нужно научиться угадывать среднестатистически наиболее вероятные ответы, что тоже является непростой задачей, требующей понимания языка.

Вариант 1 был реализован в модели [word2vec]($Efficient Estimation of Word Representations in Vector Space$) (2013). В word2vec для каждого слова в словаре обучается вектор-эмбеддинг. По сути это означает, что word2vec работает на уровне отдельных слов: модель выучивает среднестатистический смысл каждого слова. Но word2vec сам по себе не может обрабатывать фразы и длинные тексты и извлекать информацию о смысле всего текста целиком.

Идея обучения языковой модели, способной формировать высокоуровневые представления не отдельных слов, а целых фраз или текстов, является далеко не новой. Были попытки обучить языковую модель с помощью LSTM (см. [Dai et al.]($Semi-supervised Sequence Learning$) (2015), [Howard and Ruder]($Universal Language Model Fine-tuning for Text Classification$) (2018)). Однако LSTM, по-видимому, не слишком хорошо подходит для такой задачи. Несмотря на то, что сети [LSTM]($Long Short-term Memory$) (Hochreiter and Schmidhuber, 1997) были спроектированы как раз для противодействия "забыванию" предыдущей информации при поступлении новой, эта проблема все же сохраняется, как показано например в [этой недавней работе]($Slower is Better: Revisiting the Forgetting Mechanism in LSTM for Slower Information Decay$).

В 2017 году была изобретена архитектура "трансформер", на которую я делал [обзор]($Attention Is All You Need$). Трансформеры способны намного эффективнее моделировать зависимости между отдаленными друг от друга участками текста. Цитата из оригинальной работы по трансформерам:

> Learning long-range dependencies is a key challenge in many sequence transduction tasks. One key factor affecting the ability to learn such dependencies is the length of the paths forward and backward signals have to traverse in the network. The shorter these paths between any combination of positions in the input and output sequences, the easier it is to learn long-range dependencies.

Кроме того, важными особенностями трансформера является эффективная паралелизация обучения и существенный рост качества с ростом объема обучающего датасета и количества слоев в модели. Все это позволило обучать намного более эффективные языковые модели.

- **GPT** (сокращение от "Generative Pre-trained Transformer"), предложенная исследователями из OpenAI в 2018 году, использует для предобучения вариант 4 (из списка описанных выше), то есть предсказывает следующее слово в заданном тексте. В качестве архитектуры нейронной сети используется декодер трансформера.

- **BERT** (сокращение от "Bidirectional Encoder Representations from Transformers"), предложенная исследователями из Google в 2018 году, использует для предобучения вариант 2 (masked lauguage modeling) и вариант 3 (next sentence prediction). В качестве архитектуры нейронной сети используется энкодер трансформера.

Таким образом, GPT и BERT - это не архитектуры нейронных сетей, а скорее способы предобучения языковых моделей, в которых в качестве нейронной сети используется трансформер.

### Модель GPT

Статья: [Improving Language Understanding by Generative Pre-Training](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf) (Radford et al., 2018)

GPT решает задачу предсказания следующего токена в последовательности на основе предыдущих токенов. В качестве токенов авторы предлагают использовать [Byte pair encoding]($Neural Machine Translation of Rare Words with Subword Units$), то есть целые слова и наиболее часто встречающиеся части слов. Способ токенизации может влиять на качество решения, но не влияет на общий подход к задаче.

В качестве архитектуры используется модификация декодера трансформера, в которой не используется информация из энкодера (энкодер при этом вообще отсутствует). По сравнению с энкодером трансформера, декодер работает авторегрессивно, то есть первые N входных токенов однозначно определяют первые N выходных токенов (подробнее см. [здесь]($Attention Is All You Need$), раздел "Маскирование в декодере"). Иначе говоря, N-й входной токен не влияет на выходные токены от 0-го до (N-1)-го.

При обучении вход модели подается последовательность из N токенов, взятых из случайного текста. Обработав эту последовательность, модель выдает N выходных векторов, каждый из которых затем линейным слоем с softmax преобразуется в распределение вероятностей по каждому слову в словаре. Модель должна предсказывать следующий токен. Соответственно, в качестве функции потерь по каждому токену можно использовать категориальную кроссэнтропию (logloss), и затем суммировать все полученные loss'ы. Важно, что если бы модель не была авторегрессивной (например, если бы мы использовали энкодер), то она могла бы напрямую использовать (i+1)-й входной токен для предсказания i-го выходного токена, и тогда ничему бы не научилась.

В целом схема обучения устроена точно так же, как в декодере трансформера из оригинальной работы ([Vaswani et al., 2017](https://arxiv.org/abs/1706.03762)). Разница лишь в отсутствии энкодера, токенов [START] и [END].

Обученная таким образом модель способна генерировать случайные тексты. На каждом шаге генерации мы получаем распределение вероятностей выходного токена, выбираем токен с максимальной вероятностью и добавляем его ко входной последовательности ("greedy search"). Можно использовать и более сложный вариант: [beam search](https://huggingface.co/blog/how-to-generate). Здесь все, опять таки, аналогично процедуре инференса трансформера из оригинальной работы.

### Дообучение модели GPT

Языковые модели, в том числе GPT и BERT, по задумке являются лишь "полуфабрикатами", которые нужно дообучать (fine-tuning) для решения конкретной задачи. При дообучении GPT выходной слой сети создается заново, чтобы выдавать ответ в требуемом формате. Формат может быть разный, в зависимости от типа задачи (классификация, регрессия, sequence-to-sequence и т. д.).

Авторы предлагают при дообучении (файн-тюнинге) минимизировать не только основной loss, соответствующий решаемой задаче, но также и дополнительный loss, который расчитывается тем же способом, что и в предобучении: по первым i входным токенам модель должна предсказывать (i+1)-й входной токен - уже на том датасете, на котором модель дообучается. Согласно авторам, добавление этого дополнительного loss'а улучшает качество файн-тюнинга, если файн-тюнинг выполняется на большом датасете, и практически никак не влияет при файн-тюнинге на маленьких датасетах.

Кроме того, если задача такова, что при файн-тюнинге требуются дополнительные токены (например [SEP]), то их эмбеддинги также нужно инициализировать случайными значениями перед тем, как начинать файн-тюнинг, поскольку при предобучении никаких дополнительных токенов не используется.

Авторы приводят несколько конкретных примеров задач:

<img src="assets/gpt.jpg" width="700" align="center">

**Классификация.** При файн-тюнинге GPT для задачи классификации в словарь добавляются два токена: [Start] и [Extract], эмбеддинги которых инициализируются случайными весами. Входной текст разбивается на токены, в начало добавляется токен [Start], в конец добавляется токен [Extract]. Также до начала файн-тюнинга создается дополнительный полносвязный слой-классификатор со случайными весами. Среди всех выходных векторов декодера выбирается вектор, соответствующий токену [Extract] (остальные отбрасываются). Выбранный вектор преобразуется слоем-классификатором с softmax в вероятности классов.

**Textual entailment (распознавание семантического следования)**. Задача бинарной классификации, в которой нужно определить, можно ли сделать вывод о верности второй фразы (hypothesis), если известно, что верна первая (premise). В этом случае входной последовательностью является конкатенация premise и hypothesis, между ними добавляется токен [delim]. Эмбеддинг этого токена также нужно инициализировать перед началом файн-тюнинга. В остальном технически эта задача сводится к предыдущей.

**Similarity (определение сходства двух фраз)**. В этом нет явного порядка на фразах (какаяиз фраз должна идти первой). Один из возможных подходов: подаем в модель сначала конкатенацию из первой и второй фразы, затем конактенацию из второй и первой фразы. Затем берутся вектора, полученные на токене [Extract] в обоих случаях, поэлементно складываются и обрабатываются полносвязным слоем с сигмоидой, выдающим степень сходства двух фраз. Конечно, в этом случае при обратном распространении ошибки градиенты по весам модели будут рассчитаны дважды - результаты просто складываются.

В **multiple choice** все приблизительно аналогично, из экономии места опустим этот пример в данном обзоре.

### Анализ эффективности GPT и режим zero-shot

Во-первых, авторы пробуют сразу обучать модель под конкретные задачи (без предобучения). В этом случае метрика качества получается существенно хуже. Тем самым доказывается, что предобучение позволяет повысить качество решения конкретных задач.

Важно также убедиться, что все слои в модели GPT обучаются так, чтобы играть некую полезную роль в получении ответа. Для этого при файн-тюнинге авторы пробуют отбрасывать часть последних слоев модели GPT. Авторы показывают, что чем больше слоев отбрасывается, тем ниже получается метрика качества при файн-тюнинге.

Авторы формулируют следующую гипотезу: предобучение эффективно потому, что в ходе предобучения модель неявно учится решать широкий спектр NLP-задач, чтобы научиться предсказывать следующее слово в различных текстах. Чтобы проверить эту гипотезу, авторы пробуют применять языковую модель GPT в режиме zero-shot.

**Решение задачи анализа тональности в режиме zero shot**

Например, мы хотим оценить качество модели GPT на задаче анализа тональности текста (позитивная/негативная). Для этой задачи существует размеченный датасет SST-2 из пар: фраза + тональность. Мы добавляем в начало каждой фразы слово "very" и обрабатываем эту фразу моделью (без файн-тюнинга на датасете SST-2). Хотя мы подаем на вход модели законченные фразы, но модель все равно пытается достроить текст, так как была обучена на этой задаче: она выдает распределение вероятностей для слова, которое должно идти следующим. Далее мы сравниваем вероятность слов "positive" и "negative". То слово, вероятность которого окажется выше, мы и считаем оценкой тональности.

Конечно, таким образом мы получим точность ниже, чем если бы делали файн-тюнинг на датасете SST-2. Но с другой стороны, так мы показываем, что модель способна решать некоторые задачи вообще без файн-тюнинга (в режиме zero-shot), пусть и неидеально.

**Решение задачи ответов на вопросы в режиме zero shot**

Рассмотрим также задачу ответов на вопросы (question answering) на датасете [RACE](https://paperswithcode.com/dataset/race) в режиме zero-shot с помощью GPT. Выберем один из вариантов ответа и подадим на вход модели конкатенацию контекста, вопроса и текущего варианта ответа. Пусть токен T входит в состав текущего варианта ответа. Посмотрим, какое распределение вероятностей модель выдала на предыдущем токене, и чему в нем равна вероятность токена T. Проделаем то же самое для все токенов текущего варианта ответа, и перемножим полученные вероятности (или сложим их логарифмы, что то же самое). Полученное число будет считать вероятностью данного варианта ответа. Таким образом получим вероятности для всех вариантов ответа (для N вариантов потребуется N запусков модели). Далее выберем вариант с наибольшей вероятностью.

**Оценка качества GPT в режиме zero shot**

В ходе предобучения GPT авторы периодически оценивали точность модели в режиме zero-shot на разных задачах. Для каждой задачи использовались свои эвристики. Выяснилось, что в ходе обучения на всех задачах точность растет. Это подтверждает гипотезу о том, что в ходе предобучения модель GPT учится решать различные NLP-задачи.

Ниже приведен график точности в режиме zero-shot на четырех задачах, в зависимости от длительности предобучения. При этом сравнивается трансформер и LSTM. Видно, что LSTM в режиме zero-shot на всех задачах работает хуже трансформера, и особенно плохо справляется с задачей ответов на вопросы, в которой требуется анализировать связи между частями текста, далеко отстоящими друг от друга.

<img src="assets/gpt_zero_shot2.jpg" width="400" align="center">

В дальнейшем выяснилось, что подход zero-shot очень эффективен. На некоторых задачах модель вообще не обязательно дообучать, достаточно лишь "дать понять", что именно вы хотите. Например, если добавить в конце текста "TL;DR" и дополнить этот текст с помощью GPT, то модель сгенерирует сокращенный вариант текста. Более подробно про это можно почитать, например, [здесь](https://andrewmayneblog.wordpress.com/2021/04/18/the-gpt-3-zero-shot-approach/).

<img src="assets/gpt_zero_shot.jpg" width="900" align="center">

Это достаточно просто объясняется. В ходе предобучения модели уже встречались такие тексты, где после "TL;DR" шел тот же текст в сокращенном варианте. Чтобы минимизировать ошибку на этих примерах, модель научилась решать задачу суммаризации.

### Модель BERT

Статья: [BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding](https://arxiv.org/abs/1810.04805) (Devlin et al., 2018)

Авторы BERT указывают на авторегрессивность GPT как на излишнее ограничение, и в некоторых случаях даже недостаток. Модель, работающая только "слева направо" (left-to-right model) может столкнуться с трудностями при решении задач, где ответ нужно выдавать на каждом токене (token-level tasks).

**Пример 1.** Дан некий текст, по которому задан вопрос, и нужно отметить в тексте начало и конец участка, отвечающего на этот вопрос. Для этого нужно использовать информацию о вопросе. В модели, работающая "слева направо", i-й элемент выходной последовательности зависит только от первых i элементов входной последовательности, поэтому модель не сможет пометить в тексте начало и конец ответа на вопрос, еще не увидев вопроса.

**Пример 2.** Требуется решить задачу классификации токенов. Например, в задаче NER (Named Entity Recognition) нужно отметить в тексте токены, соответствующие каким-то сущностям (предмету, организации, человеку, временному промежутку и т. д.). Чтобы классифицировать i-й токен, часто требуется "заглядывать вперед".

<img src="assets/ner.png" width="600" align="center">

Авторы BERT предлагают упростить модель, убрав маскирование в self-attention. Если в GPT использовался только декодер трансформера, то в BERT используется только энкодер. На иллюстрации показано различие между архитектурами, используемыми в BERT и GPT, а также в модели ELMo.

<img src="assets/bert_gpt.jpg" width="800" align="center">

> BERT uses a bidirectional Transformer. OpenAI GPT uses a left-to-right Transformer. ELMo uses the concatenation of independently trained left-to-right and right-toleft LSTMs to generate features for downstream tasks. Among the three, only BERT representations are jointly
conditioned on both left and right context in all layers.

Однако обучить такую модель тем же способом, что и GPT, не получится. Вспомним, что GPT должна по предыдущим токенам предсказывать следующий, то есть i-й элемент выходной последовательности является вектором, который затем полносвязным слоем преобразуется в распределение вероятностей по всем словам в словаре, и таким образом предсказывается (i+1)-й элемент входной последовательности. Если убрать свойство авторегрессивности, то модель сможет "подсмотреть ответ", то есть напрямую использовать (i+1)-й входной токен для выдачи ответа на i-м выходном токене. Исходя из этого, для обучения BERT требуется другой подход.

BERT устроен как энкодер трансформера, при этом последним слоем добавлен полносвязный слой-классификатор (как в декодере трансформера и в GPT). Важно, что как в оригинальном трансформере, так и в GPT и BERT матрица весов полносвязного слоя-классификатора является транспонированной матрицей эмбеддингов. Таким образом, logit'ы будут равны скалярным произведениям выходного вектора с эмбеддингами токенов (подробнее см. обзор статьи [Attention Is All You Need]($Attention Is All You Need$), раздел "Выходной слой декодера").

Авторы обучают BERT параллельно на двух задачах:

**Задача 1** (masked language model). В словарь добавляется токен [MASK] (вектор-эмбеддинг которого, как и для всех остальных токенов, инициализируется случайными числами). При этом для разбиения текста на токены авторы используют токенизатор [WordPiece]($Googles Neural Machine Translation System: Bridging the Gap between Human and Machine Translation$), тогда как в GPT используется токенизатор BPE - впрочем, разница между ними [минимальна](https://stackoverflow.com/questions/55382596/how-is-wordpiece-tokenization-helpful-to-effectively-deal-with-rare-words-proble/55416944#55416944). В тексте, подаваемом на вход модели, случайным образом выбирается 15% токенов. Выбранные токены с 80%-й вероятностью заменяются на токен [MASK], с 10%-й вероятностью заменяются на случайное слово или с 10%-й вероятностью оставляются без изменений. Для каждого выбранного входного токена на соответствующий ему выход навешивается функция потерь: категориальная кроссэнтропия между выданным распределением вероятностей и исходным словом. Таким образом, *модель обучается восстанавливать текст, в котором 15% слов закрыты маской или изменены*.

Авторы пробовали менять вероятности маскирования токена, его замены и оставления без изменений. Результаты, полученные при переносе обучения на конкретные задачи, показаны в таблице:

<img src="assets/masking_rates.jpg" width="400" align="center">

В данной таблице "feature-based" означает заморозку модели BERT при переносе обучения, и добавление поверх нее кастомной архитектуры, решающей требуемую задачу. Авторы так описывают разницу между подходами "fine-tuning" и "feature-based":

> The feature-based approach, such as ELMo, uses task-specific architectures that include the pre-trained representations as additional features. The fine-tuning approach, such as the Generative Pre-trained Transformer (OpenAI GPT), introduces minimal task-specific parameters, and is trained on the downstream tasks by simply fine-tuning all pretrained parameters.

Как видно, на задаче NER (Named Entity Recognition) сочетание 80%-10%-10% при предобучении дает наилучший результат, а на задаче MNLI (Multi-Genre Natural Language Inference) уступает сочетанию 80%-20%-0%, но незначительно.

**Задача 2** (next sentence prediction). Для того, чтобы модель лучше научилась улавливать зависимости между отдаленными участками текста, она также обучается на следующей задаче: дан текст, склеенный из двух частей, и требуется определить, шли ли эти две части друг за другом, или же взяты из двух случайных текстов. Вводятся дополнительные токены [CLS] и [SEP]. Токен [CLS] который добавляется в начало текста, и соответствующий этому токену выходной вектор обрабатывается полносвязным слоем для бинарной классификации (остальные выходные векторы игнорируются). Две части текста, подаваемого на вход, разделяются токеном [SEP]. Также, в дополнение к позиционному эмбеддингу, вводится эмбеддинг сегментов. То есть создается еще два обучаемых вектора-эмбеддинга A и B: ко всем токенам первой части прибавляется вектор A, ко всем токенам второй части прибавляется вектор B.

<img src="assets/bert.jpg" width="600" align="center">

### Дообучение модели BERT

Как и GPT, языковая модель BERT рассматривается как заготовка, которая должна дообучаться (файн-тюниться) для решения практических задач. Авторы рассматривают разные варианты файн-тюнинга BERT на различных типах задах и приводят следующую иллюстрацию:

<img src="assets/bert3.jpg" width="600" align="center">

Далее мы рассмотрим подробнее процесс дообучения BERT на разных задачах и датасетах.

**Дообучение на задачах бенчмарка GLUE**

[GLUE]($GLUE: A Multi-Task Benchmark and Analysis Platform for Natural Language Understanding$) (General Language Understanding Evaluation) - это набор из 9 датасетов для решения различных задач, связанных с пониманием языка. Подробности о бенчмарке GLUE можно найти [здесь](https://mccormickml.com/2019/11/05/GLUE/).

<img src="assets/glue_table_condensed.png" width="1000" align="center">

На странице https://gluebenchmark.com/leaderboard можно найти наилучшие результаты, достигнутые ML-моделями на задачах бенчмарка GLUE. Там же можно найти точность, достигаемую человеком ("human baselines"). Наиболее сложными для ML-моделей оказались задачи распознавания следствий и противоречий (MNLI-mm и RTE), где по состоянию на ноябрь 2021 года человек опережает все ML-модели, хоть и незначительно. По остальным задачам ML-модели уже сравнялись или превзошли точность человека. С другой стороны, все датасеты были размечены людьми, то есть эталонные ответы так же даны людьми. Это противоречие объясняется тем, что люди - не машины и иногда совершают ошибки или бывают невнимательны, тогда как при разметке датасета все ответы перепроверялись.

Задачи бенчмарка являются задачами классификации. При дообучении BERT в начало входного текста добавляется токен [CLS], и соответствующий ему выходной вектор подается на вход полносвязному слою-классификатору. Как выяснилось, BERT превосходит GPT по всем задачам бенчмарка GLUE. В текущем лидерборде ее можно найти по названию "BERT: 24-layers, 16-heads, 1024-hidden". Многие модели, которые сейчас находятся выше в лидерборде, были созданы позднее на основе BERT.

**Дообучение на SQuAD**

Датасет SQuAD (Stanford Question Answering Dataset) содержит набор текстов и набор вопросов по каждому тексту. Для каждого вопроса указано место в тексте, где на него содержится ответ. Датасет представлен в двух версиях: 1.1 ([Rajpurkar et al., 2016]($SQuAD: 100,000+ Questions for Machine Comprehension of Text$)) и 2.0 ([Rajpurkar et al., 2018]($Know What You Dont Know: Unanswerable Questions for SQuAD$)). Вторая версия отличается от первой наличием вопросов, на которые нет ответов в тексте.

На иллюстрации, взятой из работы по BERT, слева показан процесс предобучения на задаче next sentence prediction, справа - процесс файн-тюнинга на датасете SQuAD:

<img src="assets/bert2.jpg" width="800" align="center">

При файн-тюнинге BERT на датасете SQuAD 1.1 используется тот же способ, что и в задаче next sentence prediction: вопрос и исходный текст (контекст) соединяются через токен [SEP], в начало добавляется токен [CLS], к токенам вопроса прибавляется эмбеддинг сегмента A, к токенам контекста прибавляется эмбеддинг сегмента B. В SQuAD задачей является отметить начало и конец ответа в контексте, для этого вводится два дополнительных обучаемых вектора S (start) и E (end). Пусть токенам контекста соответствуют индексы из множества $I$. Выходные векторы $\{T_i\}_{i \in I}$, соответствующие токенам контекста, скалярно умножаются на S, в результате получается набор чисел $\{S \cdot T_i\}_{i \in I}$. Если к этому набору чисел применить операцию softmax, то для каждого токена контекста мы получим вероятность того, что этот токен является началом ответа. Аналогичная операция выполняется для вектора E, и для каждого токена мы получаем числа $\{E \cdot T_i\}_{i \in I}$. Стоит отметить, что скалярное произведение можно представить как полносвязный слой с одним нейроном. Далее для каждого span'а (участка в контексте) расчитывается score. Если span начинается токеном с индексом $i$ и кончается токеном с индексом $j$, то $\text{score}_{i, j} = S \cdot T_i + E \cdot T_j$. При инференсе среди всех span'ов выбирается один с наиболее высоким score. При обучении в качестве функции потерь берется сумма логарифмов вероятностей для истинных начала и конца ответа, то есть используется кроссэнтропия (logloss).

При файн-тюнинге на датасете SQuAD 2.0 вопросы могут не иметь ответов. В этом случае в множество индексов $I$ добавляется индекс токена [CLS], и если вопрос не имеет ответа, то началом и концом span'а считается индекс токена [CLS]. При инференсе также считается score для "фейкового" span'а, который начинается и заканчивается токеном [CLS]. Также считаются scores для всех настоящих span'ов, среди которых выбирается один с максимальным score. Если score настоящего span'а превосходит score фейкового span'а на $\tau$ и более ($\tau$ - гиперпараметр), то выбирается настоящий span в качестве ответа, в противном случае выбирается вариант "нет ответа". Гиперпараметр $\tau$ подбирается так, чтобы максимизировать метрику качества F1.

**Дообучение на SWAG**

Датасет [SWAG]($SWAG: A Large-Scale Adversarial Dataset for Grounded Commonsense Inference$) (Situations With Adversarial Generations) содержит набор незаконченных фраз, для каждой из которых на выбор предлагается четыре возможных продолжения. Задача - выбрать наиболее вероятное продолжение. Авторы датасета SWAG описывают его как "task of grounded commonsense inference, unifying natural language inference and commonsense reasoning".

<img src="assets/swag.jpg" width="400" align="center">

При дообучении на SWAG осуществляется конкатенация начала фразы с одним из вариантов продолжения, в начало добавляется токен [CLS], и полученный набор токенов подается на вход модели. Выходной вектор, соответствующий токену [CLS], обрабатывается полносвязным слоем для задачи бинарной классификации, то есть осуществляется скалярное произведение выходного вектора с вектором весов слоя. В итоге для каждого из вариантов продолжения мы получаем по одному числу (logit). Полученные 4 числа с помощью операции softmax преобразуются в вероятности. Таким образом, задача сводится к мультиклассовой классификации.

**Feature-based approach**

Авторы отмечают, что не все задачи можно решить, используя в качестве архитектуры энкодер трансформера, и для некоторых все же требуется настандартная архитектура. Есть альтернативный подход: "заморозить" веса BERT и использовать ее как экстрактор признаков. Обработав с помощью BERT некий текст, мы получим набор векторов-эмбеддингов. Эти эмбеддинги содержат целостную информацию о тексте (contextual embeddings, в противоположность word-level embeddings в word2vec). Далее их можно использовать как входные данные, обучая какую-либо нестандартную архитектуру.

### Cравнение BERT и GPT и влияние размера модели на качество

На всех описанных задачах (бенчмарк GLUE, датасеты SQuAD 1.1 и 2.0, датасет SWAG) авторы BERT сравнивали модель BERT-BASE с моделью GPT (обе модели имели одинаковое количество и размеры слоев), а также обучают более тяжелую модель BERT-LARGE. На тех задачах, на которых тестировалась GPT (а именно на SWAG и GLUE), BERT-BASE ее опережает. Модель BERT-LARGE устанавливает новый state-of-the-art на 11 задачах, в том числе GLUE, SQuAD 1.1 и SQuAD 2.0.

Экспериментами, описанными в разделах 5.1 статьи о BERT, показывается, что при исключении задачи next sentence prediction из предобучения модели качество, получаемое при файн-тюнинге на конкретных задачах, падает. Также в разделе 5.2 показывается, что увеличение размера модели существенно увеличивает качество, получаемое при файн-тюнинге, даже на маленьких датасетах (3600 примеров).

> We believe that this is the first work to demonstrate convincingly that scaling to extreme model sizes also leads to large improvements on very small scale tasks, provided that the model has been sufficiently pre-trained. ... We hypothesize that when the model is fine-tuned directly on the downstream tasks and uses only a very small number of randomly initialized additional parameters, the task-specific models can benefit from the larger, more expressive pre-trained representations even when downstream task data is very small.
